<a href="https://colab.research.google.com/github/RYUK88/mapmymarketv1/blob/main/mapmymarket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install tensorflow opencv-python-headless pandas

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
import cv2
import numpy as np
import pandas as pd
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)  # Force remount to ensure latest files

# Install required libraries (if not already installed)
!pip install tensorflow opencv-python-headless pandas

# Load the pre-trained MobileNetV2 model
model = MobileNetV2(weights='imagenet')

# Label mapping to match predicted labels to inventory labels
label_mapping = {
    'Jersey': 'Shirt',
    'T Shirt': 'Shirt',
    'Shirt': 'Shirt',
    'Sneaker': 'Shoes',
    'Running Shoe': 'Shoes',
    'Shoes': 'Shoes',
    'Sock': 'Shoes',  # Mapping 'Sock' to 'Shoes' for this example
    'Watch': 'Watch',
    'Wristwatch': 'Watch',
    'Wall Clock': 'Watch'  # Mapping 'Wall Clock' to 'Watch' for this example
}

# Function to preprocess and predict the item in the image
def predict_item(image_path):
    try:
        img = cv2.imread(image_path)
        if img is None:
            return f"Error: Could not load image at {image_path}."
        img = cv2.resize(img, (224, 224))  # MobileNetV2 input size
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)
        preds = model.predict(img)
        decoded_preds = decode_predictions(preds, top=1)[0]
        item_name = decoded_preds[0][1].replace('_', ' ').title()
        return item_name
    except Exception as e:
        return f"Error: {str(e)}"

# Define the inventory file path
inventory_path = '/content/drive/MyDrive/shop_project/data/inventory.csv'

# Check if the inventory file exists
if not os.path.exists(inventory_path):
    print(f"Error: Inventory file not found at {inventory_path}. Please upload it to the correct location.")
    inventory_df = None
else:
    # Load and debug the inventory CSV
    try:
        inventory_df = pd.read_csv(inventory_path)
        print("Inventory DataFrame loaded successfully:")
        print(inventory_df.head())
        print("\nColumns in DataFrame:")
        print(inventory_df.columns.tolist())
        # Check if the expected columns are present
        expected_columns = ['Shop_Name', 'Product_Name', 'HSN_Code', 'Price', 'Address']
        if not all(col in inventory_df.columns for col in expected_columns):
            raise ValueError(f"CSV file is missing expected columns. Found: {inventory_df.columns.tolist()}")
    except Exception as e:
        print(f"Error loading or validating CSV: {str(e)}")
        inventory_df = None

# Function to find matching shops
def find_matching_shops(item_name):
    if inventory_df is None:
        return "Error: Inventory data not loaded."
    try:
        mapped_item = label_mapping.get(item_name, item_name)
        matches = inventory_df[inventory_df['Product_Name'].str.lower() == mapped_item.lower()]
        if matches.empty:
            return f"No matching shops found for '{mapped_item}'."
        else:
            return matches[['Shop_Name', 'Price', 'Address']]
    except KeyError as e:
        return f"Error: Column 'Product_Name' not found. Available columns: {inventory_df.columns.tolist()}"

# Function to process user request
def process_user_request(image_path):
    predicted_item = predict_item(image_path)
    mapped_item = label_mapping.get(predicted_item, predicted_item)
    print(f"Item Identified: {predicted_item} (Mapped to: {mapped_item})")
    matching_shops = find_matching_shops(predicted_item)
    print("Vendor Quotes and Addresses:")
    print(matching_shops)

# Test with sample images
test_images = [
    '/content/drive/MyDrive/shop_project/images/shirt.jpg',
    '/content/drive/MyDrive/shop_project/images/shoes.jpg',
    '/content/drive/MyDrive/shop_project/images/watch.jpg'
]

for image_path in test_images:
    print(f"\nTesting with {image_path.split('/')[-1]}:")
    process_user_request(image_path)

Mounted at /content/drive
Inventory DataFrame loaded successfully:
  Shop_Name Product_Name  HSN_Code   Price                 Address
0    Shop A        Shirt    6205.0   500.0             123 MG Road
1    Shop B       Jersey    6205.0   450.0        456 Brigade Road
2    Shop C        Shoes    6403.0  1200.0  ,789 Commercial Street
3    Shop D        Watch    9102.0  2000.0       101 Church Street
4       NaN          NaN       NaN     NaN                     NaN

Columns in DataFrame:
['Shop_Name', 'Product_Name', 'HSN_Code', 'Price', 'Address']

Testing with shirt.jpg:
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Item Identified: Jersey (Mapped to: Shirt)
Vendor Quotes and Addresses:
  Shop_Name  Price      Address
0    Shop A  500.0  123 MG Road

Testing with shoes.jpg:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
Item Identified: Sock (Mapped to: Shoes)
Vendor Quotes and Addresses:
  Shop_Name   Price                 Address
2    Shop C  1200.0  ,789 Commercial Street

Testing with watch.jpg:
1/